In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 13
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
# 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df, n):
    print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
    summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
    summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

    summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
    summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산

    for i in range(n):
        summary[f"{i+1}번째 값"] = df.iloc[i].values

    return summary                                                     # 요약 테이블 반환
     

In [3]:
# 2. pandas로 데이터 불러오기
import pandas as pd

# ▶ Feature 파일 불러오기
df_features = pd.read_csv('train_5.잔액정보_98이상제거.csv')

In [4]:
# ▶ Segment 파일 불러오기 (ID + Segment)
df_segment = pd.read_csv('Segment_병합.csv')

# ▶ Segment만 순서대로 붙이기 (ID 순서 동일하다는 전제)
df_features['Segment'] = df_segment['Segment'].values

# ▶ 확인
display(resumetable(df_features,3))
df_features[['ID', 'Segment']]
     

데이터셋 크기: (2400000, 74)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,잔액_일시불_B0M,int64,0,46915,998,2565,5312
3,잔액_할부_B0M,int64,0,27079,962,2390,5113
4,잔액_현금서비스_B0M,int64,0,29834,22971,0,21531
...,...,...,...,...,...,...,...
69,평잔_할부_해외_6M,int64,0,626,0,0,0
70,평잔_CA_6M,int64,0,42604,17008,0,43351
71,평잔_CA_해외_6M,int64,0,197,0,0,0
72,평잔_카드론_6M,int64,0,80666,0,0,0


,ID,Segment
0,TRAIN_000000,D
1,TRAIN_000001,E
2,TRAIN_000002,C
3,TRAIN_000003,D
4,TRAIN_000004,E
...,...,...
2399995,TRAIN_399995,E
2399996,TRAIN_399996,D
2399997,TRAIN_399997,C
2399998,TRAIN_399998,E


In [5]:

from sklearn.preprocessing import LabelEncoder

# 1. Segment를 숫자로 변환
le = LabelEncoder()
df_features['Segment_encoded'] = le.fit_transform(df_features['Segment'])

# 2. 숫자형 컬럼만 선택
numeric_cols = df_features.select_dtypes(include=['number']).columns.drop('Segment_encoded', errors='ignore')

# 3. 상관계수 계산
correlations = df_features[numeric_cols].corrwith(df_features['Segment_encoded'])

# ✅ 절댓값 적용 여부 → 아래 한 줄 주석처리로 토글
correlations = correlations.abs()

# 4. 정렬
correlations_sorted = correlations.sort_values(ascending=False)

# 5. 출력 형식
pd.set_option('display.float_format', lambda x: f'{x:.6f}')
pd.set_option('display.max_rows', None)

# 6. 출력
display(correlations_sorted)

평잔_일시불_6M         0.447967
월중평잔_일시불_B0M      0.444621
월중평잔_일시불          0.441527
평잔_일시불_3M         0.434514
잔액_일시불_B0M        0.417354
잔액_일시불_B1M        0.374244
잔액_일시불_B2M        0.373494
평잔_일시불_해외_6M      0.312756
평잔_일시불_해외_3M      0.292834
월중평잔              0.287289
평잔_6M             0.281497
평잔_CA_6M          0.273914
평잔_3M             0.268472
평잔_CA_3M          0.267652
월중평잔_CA           0.266320
잔액_현금서비스_B2M      0.265487
잔액_현금서비스_B0M      0.265011
잔액_현금서비스_B1M      0.264744
잔액_할부_B0M         0.264292
연체원금_최근           0.264225
연체일수_최근           0.264225
월중평잔_CA_B0M       0.264076
평잔_할부_6M          0.250312
잔액_할부_무이자_B0M     0.249776
월중평잔_할부_B0M       0.234879
평잔_할부_3M          0.233328
월중평잔_할부           0.232085
잔액_할부_B1M         0.228537
잔액_할부_B2M         0.228411
잔액_할부_유이자_B0M     0.146062
평잔_RV일시불_해외_6M    0.121709
평잔_RV일시불_6M       0.120489
월중평잔_RV일시불        0.120210
평잔_RV일시불_해외_3M    0.117664
평잔_RV일시불_3M       0.117160
RV_최대잔액_R12M      0.075934
RV_최대잔액_R6M       0.072831
R

In [14]:

# 7. 상관계수 0.35 이상 컬럼만 선택
selected_cols = correlations_sorted[correlations_sorted >= 0.4].index.tolist()

# 8. 해당 컬럼만 추출
X_selected = df_features[selected_cols]

# 9. 확인
print("선택된 컬럼 수:", len(selected_cols))
display(correlations[selected_cols])

선택된 컬럼 수: 5


평잔_일시불_6M      0.447967
월중평잔_일시불_B0M   0.444621
월중평잔_일시불       0.441527
평잔_일시불_3M      0.434514
잔액_일시불_B0M     0.417354
dtype: float64

In [15]:

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# vif를 계산하기 위한 상수항을 추가한다.
vif_X = sm.add_constant(X_selected)

# vif를 계산한다.
vif = pd.DataFrame()
vif['변수'] = vif_X.columns
vif['VIF'] = [variance_inflation_factor(vif_X.values, i) for i in range(vif_X.shape[1])]
vif

,변수,VIF
0,const,1.331652
1,평잔_일시불_6M,11.052343
2,월중평잔_일시불_B0M,26.447103
3,월중평잔_일시불,25.394005
4,평잔_일시불_3M,18.616740
5,잔액_일시불_B0M,11.463875


In [17]:

# vif 높은 컬럼 하나씩 제거
vif_X = X_selected.drop([
    '월중평잔_일시불_B0M',
    '평잔_일시불_3M'	
	
    ], axis=1)

# vif를 계산하기 위해 상수항을 추가한다.
vif_X2 = sm.add_constant(vif_X)

# vif를 계산한다.
vif = pd.DataFrame()
vif['변수'] = vif_X2.columns
vif['VIF'] = [variance_inflation_factor(vif_X2.values, i) for i in range(vif_X2.shape[1])]
vif

,변수,VIF
0,const,1.331643
1,평잔_일시불_6M,6.806874
2,월중평잔_일시불,11.142465
3,잔액_일시불_B0M,6.647221
